In [1]:
import pandas as pd
import numpy as np
import os
import re
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from collections import defaultdict
from IPython.display import display
from fuzzywuzzy import fuzz, process
from ITUtils import country_conflicts_finder
from pandarallel import pandarallel

# show all columns
pd.set_option('display.max_rows', None)

In [2]:
# pick a country and import the conflicts
adm = 'D'
file = 'expanded_combined_tables_conflicts_lettersatnames.csv'
filepath = os.path.join('.', 'adm_conflicts', adm, file)
df = pd.read_csv(filepath, low_memory=False)
display(df.head())
print(df.columns)

,com_el.ntc_id,com_el.tgt_ntc_id,com_el.adm,com_el.ntwk_org,com_el.sat_name,com_el.long_nom,com_el.prov,com_el.d_rcv,com_el.st_cur,orbit.orb_id,...,TPA1.2055.475-2055.725_R,TPA1.401.89525-401.90475_R,TPA1.2237.0-2238.0_E,TPA1.2065.575-2065.825_R,TPA1.401.9501-401.9699_E,TPA1.401.8901-401.9099_R,TPA1.401.95525-401.96475_E,TPA1.401.8901-401.9099_E,TPA1.401.89525-401.90475_E,TPA1.2202.4-2203.4_E
0,119545250,NaN,D,NaN,LS-4,NaN,9.1/IA,02.02.2021,50,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,119545250,NaN,D,NaN,LS-4,NaN,9.1/IA,02.02.2021,50,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,119545250,NaN,D,NaN,LS-4,NaN,9.1/IA,02.02.2021,50,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,119545250,NaN,D,NaN,LS-4,NaN,9.1/IA,02.02.2021,50,5.0,...,NaN,1.9%RE,NaN,NaN,4.0%EE,4.0%RE,1.9%EE,4.0%EE,1.9%EE,NaN
4,119545250,NaN,D,NaN,LS-4,NaN,9.1/IA,02.02.2021,50,2.0,...,NaN,1.9%RE,NaN,NaN,4.0%EE,4.0%RE,1.9%EE,4.0%EE,1.9%EE,NaN


Index(['com_el.ntc_id', ' com_el.tgt_ntc_id', ' com_el.adm',
       ' com_el.ntwk_org', ' com_el.sat_name', ' com_el.long_nom',
       ' com_el.prov', ' com_el.d_rcv', ' com_el.st_cur', ' orbit.orb_id',
       ' orbit.nbr_sat_pl', ' orbit.apog_km', ' orbit.perig_km',
       ' orbit.op_ht_km', ' s_beam.emi_rcp', ' s_beam.beam_name',
       ' grp.grp_id', ' grp.freq_min', ' grp.freq_max', ' grp.bdwdth',
       ' grp.d_inuse', ' grp.d_reg_limit', ' grp.d_prot_eff', ' grp.f_biu',
       ' emiss.seq_no', ' emiss.pwr_ds_max', ' emiss.design_emi',
       ' carrier_fr.freq_carr', ' channel.bandwidth', ' channel.freq_min',
       ' channel.freq_max', 'tpaconflicts', 'percentoverlap',
       'TPA1.401.95525-401.96475_R', 'TPA1.401.9501-401.9699_R',
       'TPA1.2055.475-2055.725_R', 'TPA1.401.89525-401.90475_R',
       'TPA1.2237.0-2238.0_E', 'TPA1.2065.575-2065.825_R',
       'TPA1.401.9501-401.9699_E', 'TPA1.401.8901-401.9099_R',
       'TPA1.401.95525-401.96475_E', 'TPA1.401.8901-401.9099_E',

In [3]:
# function to extract the table
# Initialize pandarallel
num_logical_processors = os.cpu_count()
pandarallel.initialize(nb_workers=num_logical_processors, progress_bar=True)


def condensed_summary_table(df):
    # Columns to keep
    cols = [
        ' com_el.sat_name',
        ' s_beam.beam_name',
        ' carrier_fr.freq_carr',
        ' channel.bandwidth',
        'tpaconflicts',
        'percentoverlap'
    ]
    lookupnames = [
        'UHFUP fc=401.96MHz BW=9.5kHz',
        'UHFUP fc=401.96MHz BW=19.8kHz',
        'SUP fc=2055.6MHz BW=250kHz',
        'UHFUP fc=401.90MHz BW=9.5kHz',
        'SDN fc=2237.5MHz BW=1MHz',
        'SUP fc=2065.7MHz BW=250kHz',
        'UHFDN fc=401.96MHz BW=19.8kHz',
        'UHFUP fc=401.90MHz BW=19.8kHz',
        'UHFDN fc=401.96MHz BW=9.5kHz',
        'UHFDN fc=401.90MHz BW=19.8kHz',
        'UHFDN fc=401.90MHz BW=9.5kHz',
        'SDN fc=2202.9MHz BW=1MHz'
    ]

    df1 = df[cols].copy()
    if df1.empty:
        print("❗ DataFrame is empty")
        return pd.DataFrame()

    def functiontoapply(row):
        
        import pandas as pd  # otherwhise it doesn't work under windows 

        satname = row[' com_el.sat_name'].strip()
        beamname = row[' s_beam.beam_name'].strip()
        full_id = f"{satname} - {beamname}"

        # extract the conflict indicese and percentuals
        con = str(row['tpaconflicts']).split(':')
        percent_str = str(row['percentoverlap']).split(':')
        conflict_idxs = []
        for c in con[:-1]:
            conflict_idxs.append(int(c))
        percents = str(row['percentoverlap']).split(':')

        # no conflicts case, I still need to return the satellite name (duplicates can be removed at the end) 
        # This is also the table prototype
        if not conflict_idxs or not percents:
            data = {
                'Network': str(satname),
                'Beam': 'All',
                'TPA-1 Beam': 'No Conflict',
                'percent': 0.0,  # within this function this needs to be just a number
                'fc': 0.0,  # needed for the expanded conflict literal
                'bw': 0.0  # needed fot the expanded conflict literal
            }
            return pd.DataFrame([data])

        min_len = min(len(conflict_idxs), len(percents))
        fc = float(row[' carrier_fr.freq_carr'])
        bw = float(row[' channel.bandwidth'])  # Hz
        data = [
            {
                'Network': str(satname),
                'Beam': str(beamname),
                'TPA-1 Beam': lookupnames[conflict_idxs[i]],
                'percent': float(percents[i]) if percents[i] else 0.0,
                'fc': fc,
                'bw': bw
            }
            for i in range(min_len)
            if 0 <= conflict_idxs[i] < len(lookupnames)
        ]

        return pd.DataFrame(data)

    # Apply in parallel or serial depending on size
    # if len(df1) > 24:
    #     expanded_rows = df1.parallel_apply(functiontoapply, axis=1)
    # else:
    #     tqdm.pandas()
    #     expanded_rows = df1.apply(functiontoapply, axis=1)
    tqdm.pandas()
    expanded_rows = df1.apply(functiontoapply, axis=1)

    # Flatten the list of DataFrames
    df_expanded = pd.concat(expanded_rows.tolist(), ignore_index=True)

    if df_expanded.empty:
        print("⚠️ No valid conflict data found.")
        return pd.DataFrame()

    # Get the row with max percent per group
    df_expanded.sort_values('percent', ascending=False, inplace=True)  # sort by percent
    grouped = df_expanded.groupby(['Network', 'Beam', 'TPA-1 Beam'],
                                  as_index=False).first()  # isolate the beam--TPAbeam conflicts

    # addd the literal Overlap string
    grouped['Overlap (worst case)'] = grouped.apply(
        lambda
            row: str(f"{row.percent}% @ fc={row.fc:.3f}MHz, BW={row.bw / 1e6:.3f}MHz" if row.bw >= 1e6 else f"{row.percent}% @ fc={row.fc:.3f}MHz, BW={row.bw / 1e3:.3f}kHz") if row['TPA-1 Beam'] != 'No Conflict' else str(''),
        axis=1
    )

    # Drop the columns 'bw', 'fc', and 'percent'
    grouped = grouped.drop(columns=['bw', 'fc', 'percent'])
    
    # Drop all redundant lines
    grouped = grouped.drop_duplicates()

    # Make a hard copy of the DataFrame
    grouped_copy = grouped.copy()

    # Return the hard copy of the DataFrame
    return grouped_copy



INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [4]:
# function to append satellite names without info in the ITU database
def noinfo_appender(df, satnamesfolder, countrycode):
    filename = os.path.join('.', satnamesfolder, f'{countrycode}.txt')
    # read file rows as a list
    satnames = []
    if os.path.exists(filename):
        with open(filename, 'r') as f:
            line = f.read()
            satnames = [name.strip() for name in line.split(',')]
            
    # get all satellite names
    if not df.empty:
        satfound = df['Network'].unique().tolist()
    else:
        satfound =[]
    satnotfound = [name for name in satnames if name not in satfound]    
    # append to dataframe
    rows_to_append = []
    for sat in satnotfound:
        data = {
            'Network': sat,
            'Beam': 'All',
            'TPA-1 Beam': 'No Info',
            'Overlap (worst case)': 'No data on fc'
        }
        rows_to_append.append(data)
    
    # Append all missing rows at once
    if rows_to_append:
        df = pd.concat([df, pd.DataFrame(rows_to_append)], ignore_index=True)
    
    return df.copy()

In [5]:
# usage

pivot_df = condensed_summary_table(df)
pivot_df = noinfo_appender(pivot_df, 'satellitenames',adm)
pivot_df.to_csv('conflict_percent_pivot.csv')
display(pivot_df)
    

,Network,Beam,TPA-1 Beam,Overlap (worst case)
0,AETHER-C,All,No Conflict,
1,AETHER-C,XCBL,SDN fc=2202.9MHz BW=1MHz,"40.0% @ fc=2202.375MHz, BW=250.000kHz"
2,AETHER-C,XCBS,SDN fc=2202.9MHz BW=1MHz,"40.0% @ fc=2202.375MHz, BW=250.000kHz"
3,AETHER-C,XCCL,SDN fc=2202.9MHz BW=1MHz,"100.0% @ fc=2202.625MHz, BW=250.000kHz"
4,AETHER-C,XCCS,SDN fc=2202.9MHz BW=1MHz,"100.0% @ fc=2202.625MHz, BW=250.000kHz"
5,AETHER-K,All,No Conflict,
6,AETHER-K,VK2,SDN fc=2202.9MHz BW=1MHz,"40.0% @ fc=2202.375MHz, BW=250.000kHz"
7,AETHER-K,VK3,SDN fc=2202.9MHz BW=1MHz,"100.0% @ fc=2202.625MHz, BW=250.000kHz"
8,AETHER-K,XKBL,SDN fc=2202.9MHz BW=1MHz,"40.0% @ fc=2202.375MHz, BW=250.000kHz"
9,AETHER-K,XKBS,SDN fc=2202.9MHz BW=1MHz,"40.0% @ fc=2202.375MHz, BW=250.000kHz"


In [6]:
import os

# === CONFIG ===
tpafile = './databases/TPAtable.csv'
tablesfolder = 'countriestables'
outfolder = 'adm_conflicts'
countrieslistfile = 'countrieslist.csv'

# Load country codes
with open(countrieslistfile, 'r') as f:
    countries = f.read().strip().split(', ')
# # todo comment this
# countries = ['ARG']

# === PROCESS EACH COUNTRY ===
for ccode in countries:
    print(f"\n=== Processing {ccode} ===")

    # outfolder (must already exist)
    country_outfolder = os.path.join(outfolder, ccode)

    # Read data for the 'expanded_combined_tables_conflicts_lettersatnames.csv' file
    adm = ccode
    file = 'expanded_combined_tables_conflicts_lettersatnames.csv'
    filepath = os.path.join('.', 'adm_conflicts', adm, file)
    df = pd.read_csv(filepath, low_memory=False)

    # Generate summary pivot table
    summary_pivot = condensed_summary_table(df)
    # append noinfo
    summary_pivot = noinfo_appender(summary_pivot, 'satellitenames',ccode)
    # save
    outpath = os.path.join(country_outfolder, 'conflicts_summary_condensed.csv')
    summary_pivot.to_csv(outpath, index=True)
    print('Summary condensed saved to ', outpath)

    # Read data for the 'expanded_combined_tables_conflicts_othersatnames.csv' file
    file = 'expanded_combined_tables_conflicts_othersatnames.csv'
    filepath = os.path.join('.', 'adm_conflicts', adm, file)
    df = pd.read_csv(filepath, low_memory=False)

    # Generate summary pivot table for other satellite names
    summary_pivot_othersatnames = condensed_summary_table(df)
    outpath = os.path.join(country_outfolder, 'conflicts_summary_condensed_othersatnames.csv')
    summary_pivot_othersatnames.to_csv(outpath, index=True)

    print('Summary condensed for other satellite names saved to ', outpath)




=== Processing AFS ===
Summary condensed saved to  adm_conflicts\AFS\conflicts_summary_condensed.csv
Summary condensed for other satellite names saved to  adm_conflicts\AFS\conflicts_summary_condensed_othersatnames.csv

=== Processing ALG ===
❗ DataFrame is empty
Summary condensed saved to  adm_conflicts\ALG\conflicts_summary_condensed.csv
Summary condensed for other satellite names saved to  adm_conflicts\ALG\conflicts_summary_condensed_othersatnames.csv

=== Processing ARG ===
❗ DataFrame is empty
Summary condensed saved to  adm_conflicts\ARG\conflicts_summary_condensed.csv
Summary condensed for other satellite names saved to  adm_conflicts\ARG\conflicts_summary_condensed_othersatnames.csv

=== Processing ARS ===
Summary condensed saved to  adm_conflicts\ARS\conflicts_summary_condensed.csv
Summary condensed for other satellite names saved to  adm_conflicts\ARS\conflicts_summary_condensed_othersatnames.csv

=== Processing AUS ===
Summary condensed saved to  adm_conflicts\AUS\conflict